In [1]:
from config.features import ACTIVE_FEATURES
from torch.utils.data import DataLoader
from utils.datasets import BaseDataset, base_collate

val_dataset = BaseDataset('dhp', 826, 826, features=ACTIVE_FEATURES, max_rows=None, minus_label_col=None)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=True, collate_fn=base_collate)

D:\codes\www\utils\common.py:33: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(csv_file, usecols=columns, sep=sep)
2024-10-09 10:33:47,071 - common.py - INFO - Read dataframe from csv D:/codes/www\processed_data\dhp/dhp_6h_826.csv | sample number: 601324
2024-10-09 10:33:47,072 - common.py - INFO - sample num: 601324


In [3]:
import numpy as np
import os
import pandas as pd
import torch
from tqdm.auto import tqdm


def predict(model, test_loader, save_path=None, prefix='default', label_col='ltv3'):
    model.eval()
    # init
    result = {
        'user_type': [],
        'is_pay': [],
        'pay_probs': [],
        label_col: [],
        'p' + label_col: []
    }
    loss = []
    for batch in tqdm(test_loader, total=len(test_loader), desc='Predicting'):
        config, inputs, labels = batch
        if isinstance(labels, dict):
            labels = labels[label_col]
        # user_type
        result["user_type"] += inputs["user_type"].tolist()
        # ground
        result["is_pay"] += (labels > 0).to(torch.int32).tolist()
        result[label_col] += labels.tolist()
        # predict
        predict_result = model(batch)
        result["pay_probs"] += predict_result['prediction_p'].squeeze().tolist()
        result['p' + label_col] += predict_result['prediction_v'].squeeze().tolist()
        loss.append(predict_result['loss'].detach().item())

    result_df = pd.DataFrame(result)
    if save_path is not None:
        save_file = os.path.join(save_path, f'{prefix}_prediction.csv')
        result_df.to_csv(save_file, index=False)

    return result_df, np.mean(loss).item()

In [5]:
from utils.common import evaluate
from modeling.cmltv_model import CMLTV_Model

eval_result_list = {}
for e in range(1, 11):
    model = CMLTV_Model(features=ACTIVE_FEATURES, device='cuda:0')
    model.load_weights(rf'D:\codes\www\checkpoints\dhp\dhp_cmltv_active_seed_42_test_old\{e}.pth')
    model.eval()
    predict_df, val_loss = predict(model, val_loader)
    eval_result = evaluate(predict_df)
    print(eval_result)
    eval_result_list[e] = eval_result
    pd.DataFrame(eval_result_list).T.to_csv(os.path.join(r'D:\codes\www\checkpoints\dhp\dhp_cmltv_active_seed_42_test_old', 'epoch_evaluation_new.csv'))

2024-10-09 10:38:58,427 - cmltv_model.py - INFO - base model info: {'hidden_dim': 172, 'params': {'Total': 388382, 'Trainable': 388382}, 'device': 'cuda:0'}


Predicting:   0%|          | 0/588 [00:00<?, ?it/s]

2024-10-09 10:43:07,554 - cmltv_model.py - INFO - base model info: {'hidden_dim': 172, 'params': {'Total': 388382, 'Trainable': 388382}, 'device': 'cuda:0'}


{'auc': 0.8564105005222603, 'gini_by_pltv': 0.6970724979183, 'gini_by_p': 0.6837490185200173, 'rmse': 19.841061821515815, 'nrmse': 0.0025102557972565558, 'mae': 0.4203760231544224, 'nmae': 5.318522560152105e-05, 'top_0.01_p_by_pltv': 0.09462830533843339, 'top_0.01_r_by_pltv': 0.134994068801898, 'top_0.03_p_by_pltv': 0.06092355452076057, 'top_0.03_r_by_pltv': 0.2607354685646501, 'top_0.05_p_by_pltv': 0.04849331470764318, 'top_0.05_r_by_pltv': 0.3459074733096085, 'top_0.1_p_by_pltv': 0.0347568682232422, 'top_0.1_r_by_pltv': 0.4958481613285884, 'top_0.01_p_by_p': 0.10793281224014635, 'top_0.01_r_by_p': 0.15397390272835113, 'top_0.03_p_by_p': 0.07511502854925439, 'top_0.03_r_by_p': 0.32147093712930014, 'top_0.05_p_by_p': 0.05907004589902215, 'top_0.05_r_by_p': 0.4213523131672598, 'top_0.1_p_by_p': 0.03947981108228564, 'top_0.1_r_by_p': 0.563226571767497, 'top_amount_recall_by_pltv': {}, 'top_0.01_amount_recall_by_pltv': 34007.1, 'top_0.05_amount_recall_by_pltv': 71231.1, 'top_0.1_amount_re

D:\codes\www\modeling\cmltv_model.py:131: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))


...

2024-10-09 11:14:52,007 - cmltv_model.py - INFO - base model info: {'hidden_dim': 172, 'params': {'Total': 388382, 'Trainable': 388382}, 'device': 'cuda:0'}


{'auc': 0.8690975945948116, 'gini_by_pltv': 0.7233292631707144, 'gini_by_p': 0.7126866321474445, 'rmse': 20.09502576498983, 'nrmse': 0.002542386862979483, 'mae': 0.44413335136868165, 'nmae': 5.619096044644251e-05, 'top_0.01_p_by_pltv': 0.11159155163811742, 'top_0.01_r_by_pltv': 0.15919335705812573, 'top_0.03_p_by_pltv': 0.07029214479738345, 'top_0.03_r_by_pltv': 0.30083036773428234, 'top_0.05_p_by_pltv': 0.053981241269207746, 'top_0.05_r_by_pltv': 0.3850533807829182, 'top_0.1_p_by_pltv': 0.037184859974722276, 'top_0.1_r_by_pltv': 0.5304863582443654, 'top_0.01_p_by_p': 0.12888741061034426, 'top_0.01_r_by_p': 0.18386714116251482, 'top_0.03_p_by_p': 0.0835412162536726, 'top_0.03_r_by_p': 0.3575326215895611, 'top_0.05_p_by_p': 0.06326082618239873, 'top_0.05_r_by_p': 0.4512455516014235, 'top_0.1_p_by_p': 0.0409765183263487, 'top_0.1_r_by_p': 0.5845788849347568, 'top_amount_recall_by_pltv': {}, 'top_0.01_amount_recall_by_pltv': 37809.1, 'top_0.05_amount_recall_by_pltv': 74548.1, 'top_0.1_amo

D:\codes\www\modeling\cmltv_model.py:131: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))


Predicting:   0%|          | 0/588 [00:00<?, ?it/s]

{'auc': 0.8687929473831073, 'gini_by_pltv': 0.712168877867404, 'gini_by_p': 0.7055789090978734, 'rmse': 20.013032207626882, 'nrmse': 0.00253201318416332, 'mae': 0.4205808316175937, 'nmae': 5.32111376034405e-05, 'top_0.01_p_by_pltv': 0.10560452353234659, 'top_0.01_r_by_pltv': 0.1506524317912218, 'top_0.03_p_by_pltv': 0.06724319529907423, 'top_0.03_r_by_pltv': 0.2877817319098458, 'top_0.05_p_by_pltv': 0.05251779418612386, 'top_0.05_r_by_pltv': 0.3746144721233689, 'top_0.1_p_by_pltv': 0.03668595756003459, 'top_0.1_r_by_pltv': 0.5233689205219454, 'top_0.01_p_by_p': 0.12855479793780142, 'top_0.01_r_by_p': 0.1833926453143535, 'top_0.03_p_by_p': 0.08304229724485836, 'top_0.03_r_by_p': 0.3553973902728351, 'top_0.05_p_by_p': 0.06226302135302335, 'top_0.05_r_by_p': 0.4441281138790036, 'top_0.1_p_by_p': 0.04071043703851527, 'top_0.1_r_by_p': 0.5807829181494661, 'top_amount_recall_by_pltv': {}, 'top_0.01_amount_recall_by_pltv': 39783.1, 'top_0.05_amount_recall_by_pltv': 73876.1, 'top_0.1_amount_re